# Apriory Algorithm

In [1]:
import re
import operator
from time import time
from collections import defaultdict

In [2]:
def normalize_group(*args):
    return str(sorted(args))


def generate_pairs(*args):
    pairs = []
    for idx_1 in range(len(args) - 1):
        for idx_2 in range(idx_1 + 1, len(args)):
            pairs.append(normalize_group(args[idx_1], args[idx_2]))
    return pairs


def generate_triples(*args):
    triples = []
    for idx_1 in range(len(args) - 2):
        for idx_2 in range(idx_1 + 1, len(args) - 1):
            for idx_3 in range(idx_2 + 1, len(args)):
                triples.append(normalize_group(args[idx_1], args[idx_2], args[idx_3]))
    return triples


def generate_fourfolds(*args):
    fourfolds = []
    for idx_1 in range(len(args) - 3):
        for idx_2 in range(idx_1 + 1, len(args) - 2):
            for idx_3 in range(idx_2 + 1, len(args) - 1):
                for idx_4 in range(idx_3 + 1, len(args)):
                    fourfolds.append(normalize_group(args[idx_1], args[idx_2], args[idx_3], args[idx_4]))
    return fourfolds


def separator(regularities):
    regularities_ID_R = []
    single_unique_terms = set()
    
    for reg in regularities:
        terms = re.split('\[\'|\', \'|\'\]', reg[0])[1:-1]
        for term in terms: single_unique_terms.add(term)
        
    for unique_term in single_unique_terms:
        ID_R = re.split('Item_|: ', unique_term)[1:]
        regularities_ID_R.append(list(map(int, ID_R)))
            
    return single_unique_terms, regularities_ID_R

In [3]:
def apriory_algorithm(THRESHOLD=30, file='../datasets/filter_2/file_100K.txt'):
    tic = time()
    THRESHOLD = THRESHOLD
    
    frequent_items = set()
    frequent_pairs = set()
    frequent_triples = set()
    frequent_fourfold = set()
    frequent_fivefold = set()
    
    item_counts = defaultdict(int)
    pair_counts = defaultdict(int)
    triple_counts = defaultdict(int)
    fourfold_counts = defaultdict(int)
    fivefold_counts = defaultdict(int)
    
    # read in the data
    with open(file) as f:
        lines = f.readlines()
    f.close()
    

    # FIRST PASS ----------------------------------------------------------------------------------

    # first pass - find candidate items
    for line in lines:
        for item in line.split(', '):
            item_counts[item] += 1

    # first pass - find frequent items
    for key in item_counts:
        if item_counts[key] > THRESHOLD:
            frequent_items.add(key)
            
    print('There are {0} unique items, {1} of which are frequent'.format(len(item_counts), len(frequent_items)))
    
    
    # SECOND PASS ----------------------------------------------------------------------------------

    # second pass - find candidate pairs
    # when building candidate pairs, only consider frequent items
    for line in lines:
        items = line.split(', ')
        for idx_1 in range(len(items) - 1):
            if items[idx_1] not in frequent_items:
                continue
            for idx_2 in range(idx_1 + 1, len(items)):
                if items[idx_2] not in frequent_items:
                    continue
                pair = normalize_group(items[idx_1], items[idx_2]) # [a, b] is the same as [b, a]
                pair_counts[pair] += 1

    # second pass - find frequent pairs
    for key in pair_counts:
        if pair_counts[key] > THRESHOLD:
            frequent_pairs.add(key)

    print('There are {0} candidate pairs, {1} of which are frequent'.format(len(pair_counts), len(frequent_pairs)))
    
    
    # THIRD PASS ----------------------------------------------------------------------------------

    # third pass - find candidate triples
    # when building candidate triples, only consider frequent items and pairs
    if len(frequent_pairs) != 0: 
        for line in lines:
            items = line.split(', ')
            for idx_1 in range(len(items) - 2):
                if items[idx_1] not in frequent_items:
                    continue
                for idx_2 in range(idx_1 + 1, len(items) - 1):
                    first_pair = normalize_group(items[idx_1], items[idx_2])
                    if items[idx_2] not in frequent_items or first_pair not in frequent_pairs:
                        continue
                    for idx_3 in range(idx_2 + 1, len(items)):
                        if items[idx_3] not in frequent_items:
                            continue
                        # now check that all pairs are frequent, since this is a precondition to being a frequent triple
                        pairs = generate_pairs(items[idx_1], items[idx_2], items[idx_3])
                        if any(pair not in frequent_pairs for pair in pairs):
                            continue
                        triple = normalize_group(items[idx_1], items[idx_2], items[idx_3])
                        triple_counts[triple] += 1

        # third pass - find frequent triples
        for key in triple_counts:
            if triple_counts[key] > THRESHOLD:
                frequent_triples.add(key)

        print('There are {0} candidate triples, {1} of which are frequent'.format(len(triple_counts), len(frequent_triples)))
    
    

    # FOURTH PASS ----------------------------------------------------------------------------------
        
    # fourth pass - find candidate fourfold
    # when building candidate fourfold, only consider frequent items, pairs, and triples
    if len(frequent_triples) != 0:
        for line in lines:
            items = line.split(', ')
            for idx_1 in range(len(items) - 3):
                if items[idx_1] not in frequent_items: # first item must be frequent
                    continue
                for idx_2 in range(idx_1 + 1, len(items) - 2):
                    first_pair = normalize_group(items[idx_1], items[idx_2])
                    if items[idx_2] not in frequent_items or first_pair not in frequent_pairs:
                        continue
                    for idx_3 in range(idx_2 + 1, len(items) - 1):
                        second_pair = normalize_group(items[idx_1], items[idx_2], items[idx_3])
                        if items[idx_3] not in frequent_items or second_pair not in frequent_triples:
                            continue   
                        for idx_4 in range(idx_3 + 1, len(items)):
                            if items[idx_4] not in frequent_items:
                                continue
                            # now check that all triples are frequent, since this is a precondition to being a frequent fourfold
                            triples = generate_triples(items[idx_1], items[idx_2], items[idx_3], items[idx_4])
                            if any(trip not in frequent_triples for trip in triples):
                                continue
                            fourfold = normalize_group(items[idx_1], items[idx_2], items[idx_3], items[idx_4])
                            fourfold_counts[fourfold] += 1

        # fourth pass - find frequent fourfold
        for key in fourfold_counts:
            if fourfold_counts[key] > THRESHOLD:
                frequent_fourfold.add(key)

        print('There are {0} candidate fourfold, {1} of which are frequent'.format(len(fourfold_counts), len(frequent_fourfold)))
    
    
    # FIFTH PASS -----------------------------------------

    # fifth pass - find candidate fivefold
    # when building candidate fivefold, only consider frequent items, pairs, triples, and fourfold
    if len(frequent_fourfold) != 0:
        for line in lines:
            items = line.split(', ')
            for idx_1 in range(len(items) - 4):
                if items[idx_1] not in frequent_items:
                    continue
                for idx_2 in range(idx_1 + 1, len(items) - 3):
                    first_pair = normalize_group(items[idx_1], items[idx_2])
                    if items[idx_2] not in frequent_items or first_pair not in frequent_pairs:
                        continue
                    for idx_3 in range(idx_2 + 1, len(items) - 2):
                        second_pair = normalize_group(items[idx_1], items[idx_2], items[idx_3])
                        if items[idx_3] not in frequent_items or second_pair not in frequent_triples:
                            continue
                        for idx_4 in range(idx_3 + 1, len(items) - 1):
                            third_pair = normalize_group(items[idx_1], items[idx_2], items[idx_3], items[idx_4])
                            if items[idx_4] not in frequent_items or third_pair not in frequent_fourfold:
                                continue
                            for idx_5 in range(idx_4 + 1, len(items)):
                                if items[idx_5] not in frequent_items:
                                    continue
                                # now check that all fourfold are frequent, since this is a precondition to being a frequent fivefold
                                fourfolds = generate_fourfolds(items[idx_1], items[idx_2], items[idx_3], items[idx_4], items[idx_5])
                                if any(four not in frequent_fourfold for four in fourfolds):
                                    continue
                                fivefold = normalize_group(items[idx_1], items[idx_2], items[idx_3], items[idx_4], items[idx_5])
                                fivefold_counts[fivefold] += 1

        # fifth pass - find frequent fivefold
        for key in fivefold_counts:
            if fivefold_counts[key] > THRESHOLD:
                frequent_fivefold.add(key)

        print('There are {0} candidate fivefold, {1} of which are frequent'.format(len(fivefold_counts), len(frequent_fivefold))) 

    
    # VIEW OUR RESULTS -------------------------------------------------------------------------------
    print('--------------')
    
    if len(frequent_fivefold) != 0:
        fold_counts = fivefold_counts
    elif len(frequent_fourfold) != 0:
        fold_counts = fourfold_counts     
    elif len(frequent_triples) != 0:
        fold_counts = triple_counts     
    elif len(frequent_pairs) != 0:
        fold_counts = pair_counts
    else:
        fold_counts = item_counts
    
    frequent_folds = { k: v for k, v in fold_counts.items() if v > THRESHOLD }         # Get frequent fourfolds + counts
    sorted_frequent_folds = sorted(frequent_folds.items(), key=operator.itemgetter(1)) # Sorted by count of Supports

    for entry in sorted_frequent_folds:
        print('{0}: {1}'.format(entry[0], entry[1]))
        
    print('Preprocessing time: {} mins!'.format(int((time() - tic) / 60.0)))
        
    return sorted_frequent_folds

In [4]:
# MovieLens 100K
# 1th PASS: There are 7249 unique items, 923 of which are frequent
# 2th PASS: There are 411802 candidate pairs, 3410 of which are frequent
# 3th PASS: There are 28879 candidate triples, 771 of which are frequent
# 4th PASS: There are 465 candidate fourfold, 118 of which are frequent
# 5th PASS: There are 9 candidate fivefold, 7 of which are frequent         ****

# MovieLens 1M 300
# 1th PASS: There are 18069 unique items, 598 of which are frequent
# 2th PASS: There are 178208 candidate pairs, 659 of which are frequent
# 3th PASS: There are 2823 candidate triples, 151 of which are frequent
# 4th PASS: There are 35 candidate fourfold, 6 of which are frequent         ****

# MovieLens 1M 430
# 1th PASS: There are 18069 unique items, 280 of which are frequent
# 2th PASS: There are 38955 candidate pairs, 167 of which are frequent
# 3th PASS: There are 399 candidate triples, 11 of which are frequent        ****
# 4th PASS: There are 1 candidate fourfold, 0 of which are frequent

# MovieLens 1M 500
# 1th PASS: There are 18069 unique items, 198 of which are frequent
# 2th PASS: There are 19435 candidate pairs, 87 of which are frequent
# 3th PASS: There are 163 candidate triples, 3 of which are frequent         ****